In [ ]:
import pymupdf
import PIL
from PIL import Image, ImageOps
import numpy as np
import PathStorage
import os
import pandas as pd
import scipy.ndimage as nd


In [ ]:
def split(array, start, end, left = 0, right = 1):
    start_pos = int(array.shape[0] * start)
    end_pos = int(array.shape[0] * end)
    left_pos = int(array.shape[1] * left)
    right_pos = int(array.shape[1] * right)
    return array[start_pos:end_pos,left_pos:right_pos,:]

def isolate_grey(array):
    array[array < 90] = 255
    array[array < 200] = 0
    array = nd.median_filter(array, 3)
    array[array < 220] = 0
    array = nd.maximum_filter(array, 3)
    array[array > 1] = 255
    array = nd.gaussian_filter(array, 3)
    array = nd.maximum_filter(array, 3)
    array[array < 200] = 0
    array[array > 1] = 255
    array = array.astype(np.int32)
    return array

In [ ]:
png_index = 0


for folder in os.listdir(PathStorage.completed_path):
    env_path = os.path.join(PathStorage.completed_path, folder, "Environment")
    conversion_path = os.path.join(env_path, "document_data.csv")
    df = pd.read_csv(conversion_path, index_col=0)
    loc_mask = df["DateLocation"] != "e"
    date_mask = df["DateType"] == 2
    df = df[loc_mask & date_mask]
    
    for index, row in df.iterrows():
        if(png_index in range(0, 1860)):
            png_index += 2
            continue
        try:
            old_path_parts = row["Path"].split("\\")
            reconstructed = os.path.join(env_path, old_path_parts[-3], old_path_parts[-2], old_path_parts[-1])

            pos_list = row["DateLocation"].split(",")
            if(len(pos_list) == 3):
                pos_list[1] = pos_list[0] + pos_list[1]
                pos_list.pop(0)
            if(len(pos_list) == 2):
                pos_list.append(float(pos_list[0]) + 500)
                pos_list.append(float(pos_list[1]) + 100)


            doc = pymupdf.open(reconstructed)
            page = doc.load_page(0)
            pix = page.get_pixmap(dpi=300)
            doc.close()

            image = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
            array = np.asarray(image, dtype=np.int32)


            sliced = split(np.copy(array), .0, .15)

            pos_list[2] = min(float(pos_list[2]), sliced.shape[1]- 1)
            pos_list[3] = min(float(pos_list[3]), sliced.shape[0] - 1)
            pos_list = [int(x) for x in pos_list]

            #sliced = sliced[pos_list[0]:pos_list[2], pos_list[1]: pos_list[3],:]
            sliced = sliced[pos_list[1]: pos_list[3], pos_list[0]:pos_list[2], :]

            # No Mod

            result_sliced_path = os.path.join(PathStorage.text_dataset, "{:05d}".format(png_index) + "_" + str(row["Date"]) + ".png")

            im = Image.fromarray((sliced).astype(np.uint8))
            im.save(result_sliced_path)

            png_index += 1
                
            # Grey Filter
            
            isolate_grey(sliced)
            result_sliced_path = os.path.join(PathStorage.text_dataset, "{:05d}".format(png_index) + "_" + str(row["Date"]) + ".png")

            im = Image.fromarray((sliced).astype(np.uint8))
            im.save(result_sliced_path)

            png_index += 1

            if(png_index % 20 == 0):
                print(png_index)
        except:
            print("ERROR" + str(pos_list) + " " + str(env_path) + " " + str(index))
            


    

In [ ]:
with open(PathStorage.text_dataset_file, 'w') as s:
    s.seek(0)
    for i in os.listdir(PathStorage.text_dataset):
        s.writelines(os.path.join("test",i) + "\t" + i.split("_")[1].split(".")[0] + "\n")
    s.close()